In [1]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.utils import train_test_split_edges


In [2]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


/Users/sadiaanjum/miniconda3/envs/py_env/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
data

Data(x=[3327, 3703], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

In [3]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x2 =  self.conv_mu(x, edge_index)
        x3 = self.conv_logstd(x, edge_index)
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [4]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [5]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)

In [6]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [8]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6015, AP: 0.6384
Epoch: 002, AUC: 0.6018, AP: 0.6365
Epoch: 003, AUC: 0.6048, AP: 0.6379
Epoch: 004, AUC: 0.6069, AP: 0.6389
Epoch: 005, AUC: 0.6090, AP: 0.6413
Epoch: 006, AUC: 0.6097, AP: 0.6426
Epoch: 007, AUC: 0.6096, AP: 0.6435
Epoch: 008, AUC: 0.6098, AP: 0.6445
Epoch: 009, AUC: 0.6095, AP: 0.6451
Epoch: 010, AUC: 0.6093, AP: 0.6458
Epoch: 011, AUC: 0.6091, AP: 0.6465
Epoch: 012, AUC: 0.6092, AP: 0.6474
Epoch: 013, AUC: 0.6092, AP: 0.6481
Epoch: 014, AUC: 0.6099, AP: 0.6496
Epoch: 015, AUC: 0.6115, AP: 0.6515
Epoch: 016, AUC: 0.6124, AP: 0.6523
Epoch: 017, AUC: 0.6133, AP: 0.6530
Epoch: 018, AUC: 0.6116, AP: 0.6533
Epoch: 019, AUC: 0.6070, AP: 0.6523
Epoch: 020, AUC: 0.6009, AP: 0.6511
Epoch: 021, AUC: 0.5918, AP: 0.6479
Epoch: 022, AUC: 0.5849, AP: 0.6451
Epoch: 023, AUC: 0.5799, AP: 0.6421
Epoch: 024, AUC: 0.5794, AP: 0.6418
Epoch: 025, AUC: 0.5810, AP: 0.6430
Epoch: 026, AUC: 0.5855, AP: 0.6465
Epoch: 027, AUC: 0.5907, AP: 0.6497
Epoch: 028, AUC: 0.5998, AP:

In [13]:
z = model.encode(x, train_pos_edge_index)

final = model.decode(z, train_pos_edge_index)

In [14]:
final

tensor([0.5146, 0.9221, 0.9554,  ..., 0.6987, 0.5864, 0.8571],
       grad_fn=<SigmoidBackward0>)

In [15]:
model.encode

<bound method VGAE.encode of VGAE(
  (encoder): VariationalGCNEncoder(
    (conv1): GCNConv(3703, 4)
    (conv_mu): GCNConv(4, 2)
    (conv_logstd): GCNConv(4, 2)
  )
  (decoder): InnerProductDecoder()
)>